In [1]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams['font.family'] = 'Malgun Gothic'
import datetime
import time

In [2]:
def get_gu_dong(df):
    df = df.copy()
    
    df['구'] = df['지번주소'].apply(lambda x: x.split(' ')[1])
    df['동'] = df['지번주소'].apply(lambda x: x.split(' ')[2])
    
    return df

In [3]:
bulktrade_basedir = './국토교통부_실거래가_공개시스템/집값분석/일괄구매/10호이상/'

In [4]:
date_today = pd.to_datetime(datetime.datetime.now())
target_period = 6 # in months

In [5]:
yunrip_bulk_df = pd.read_csv(bulktrade_basedir+'연립다세대_일괄구매_전체.csv')
yunrip_bulk_df['계약날짜'] = pd.to_datetime(yunrip_bulk_df['계약날짜'])
yunrip_bulk_df = yunrip_bulk_df[yunrip_bulk_df['계약날짜']  >= (date_today - pd.DateOffset(months=target_period))]
yunrip_bulk_df = yunrip_bulk_df[
    (yunrip_bulk_df['소유자'] == '서***~')
    |(yunrip_bulk_df['소유자'] == '한***~')
].reset_index(drop=True)
yunrip_bulk_df = get_gu_dong(yunrip_bulk_df)
print(yunrip_bulk_df.shape)
yunrip_bulk_df.head()

(2641, 16)


,지번주소,도로명,건물(단지)명,전용면적(㎡),대지권면적(㎡),거래금액(만원),층,건축년도,계약날짜기준_건물연식,계약날짜,단가(만원/㎡),건물연식,부동산유형,소유자,구,동
0,서울특별시 강동구 길동 349-9,천중로48길 9-1,강동서영스윗홈,32.04,19.05,26850,5,2020.0,0.0,2020-12-28,838.014981,1.0,연립다세대,서***~,강동구,길동
1,서울특별시 강동구 길동 349-9,천중로48길 9-1,강동서영스윗홈,23.16,13.77,19700,5,2020.0,0.0,2020-12-28,850.604491,1.0,연립다세대,서***~,강동구,길동
2,서울특별시 강동구 길동 349-9,천중로48길 9-1,강동서영스윗홈,38.84,23.09,33050,4,2020.0,0.0,2020-12-28,850.926880,1.0,연립다세대,서***~,강동구,길동
3,서울특별시 강동구 길동 349-9,천중로48길 9-1,강동서영스윗홈,32.04,19.05,27350,4,2020.0,0.0,2020-12-28,853.620474,1.0,연립다세대,서***~,강동구,길동
4,서울특별시 강동구 길동 349-9,천중로48길 9-1,강동서영스윗홈,32.04,19.05,27800,2,2020.0,0.0,2020-12-28,867.665418,1.0,연립다세대,서***~,강동구,길동


In [6]:
yunrip_bulk_df['지번별_전용면적별_평균단가'] = yunrip_bulk_df[['지번주소', '건축년도', '계약날짜', '전용면적(㎡)', '단가(만원/㎡)']].groupby(['지번주소', '건축년도', '계약날짜', '전용면적(㎡)']).transform('mean')['단가(만원/㎡)']

In [7]:
yunrip_selected_df = yunrip_bulk_df[['지번주소', '건축년도', '건물연식', '계약날짜', '계약날짜기준_건물연식', '전용면적(㎡)', '지번별_전용면적별_평균단가', '부동산유형', '소유자']].drop_duplicates().reset_index(drop=True)
print(yunrip_selected_df.shape)
yunrip_selected_df.head()

(1300, 9)


,지번주소,건축년도,건물연식,계약날짜,계약날짜기준_건물연식,전용면적(㎡),지번별_전용면적별_평균단가,부동산유형,소유자
0,서울특별시 강동구 길동 349-9,2020.0,1.0,2020-12-28,0.0,32.04,856.741573,연립다세대,서***~
1,서울특별시 강동구 길동 349-9,2020.0,1.0,2020-12-28,0.0,23.16,850.604491,연립다세대,서***~
2,서울특별시 강동구 길동 349-9,2020.0,1.0,2020-12-28,0.0,38.84,852.857878,연립다세대,서***~
3,서울특별시 강동구 길동 349-9,2020.0,1.0,2020-12-28,0.0,36.84,858.668114,연립다세대,서***~
4,서울특별시 강동구 성내동 529-2,2020.0,1.0,2020-12-28,0.0,34.44,871.080139,연립다세대,서***~


In [8]:
yunrip_selected_df = yunrip_selected_df.sort_values(['지번주소']).reset_index(drop=True)
yunrip_selected_df.to_excel(bulktrade_basedir + '최근_6개월_SH_LH_거래사례.xlsx', index=False)

In [ ]:
yunrip_selected_df['지번별_평균단가'] = yunrip_selected_df[['지번주소', '건축년도', '계약날짜', '지번별_전용면적별_평균단가']].groupby(['지번주소', '건축년도', '계약날짜']).transform('mean')['지번별_전용면적별_평균단가']

In [ ]:
yunrip_selected_df = yunrip_selected_df.drop(columns=['전용면적(㎡)', '지번별_전용면적별_평균단가']).drop_duplicates().reset_index(drop=True)
print(yunrip_selected_df.shape)
yunrip_selected_df.head()

In [ ]:
yunrip_selected_df['구'] = yunrip_selected_df['지번주소'].apply(lambda x: x.split(' ')[1])
yunrip_selected_df['동'] = yunrip_selected_df['지번주소'].apply(lambda x: x.split(' ')[2])
yunrip_selected_df['구동'] = yunrip_selected_df['구'] + '_' + yunrip_selected_df['동']

In [ ]:
basedir = './국토교통부_실거래가_공개시스템/땅값분석/'
yunrip_df = pd.read_csv(basedir + 'yunrip_25.csv')
commerce_df = pd.read_csv(basedir + 'commerce_25.csv')
land_df = pd.read_csv(basedir + 'land.csv')

In [ ]:
common_cols = ['시군구', '번지', '용도지역', '도로조건', '계약날짜', '대지면적(㎡)', '거래금액(만원)', '단가(㎡)', '단가(평)', '부동산유형', '건물연식']
common_df = pd.concat([yunrip_df[common_cols], commerce_df[common_cols], land_df[[col for col in land_df.columns if col not in ['지목', '지분구분', '해제사유발생일']]]])
common_df['계약날짜'] = pd.to_datetime(common_df['계약날짜'], format='%Y-%m-%d')
print(common_df.shape)
common_df.head()

In [ ]:
common_df['구'] = common_df['시군구'].apply(lambda x: x.split(' ')[1])
common_df['동'] = common_df['시군구'].apply(lambda x: x.split(' ')[2])

In [ ]:
yunrip_bulk_df['구동'] = yunrip_bulk_df['구'] + '_' + yunrip_bulk_df['동']

In [ ]:
summary_df = pd.DataFrame({'지역':[], '2종일반중간값':[], '집값중간값':[]})

count = 0
for gudong in tqdm(yunrip_selected_df['구동'].unique()):
    gu = gudong.split('_')[0]
    dong = gudong.split('_')[1]
    
    target_gu = gu
    target_dong = dong
    target_purpose = '2종일반'
    
    target_df = common_df[
        (common_df['구'] == target_gu)
        &(common_df['동'] == target_dong)
        &(common_df['용도지역'].str.contains(target_purpose))
        &(common_df['계약날짜'] >= (date_today - pd.DateOffset(months=target_period)))
        &(common_df['단가(㎡)'] > 250)
    ].sort_values(['번지'])
    
    yunrip_target_df = yunrip_selected_df[
        (yunrip_selected_df['구동'] == gudong)
    ]
    
    summary_df.loc[count, '지역'] = gudong
    summary_df.loc[count, '2종일반중간값'] = target_df['단가(㎡)'].median()
    summary_df.loc[count, '집값중간값'] = yunrip_target_df['지번별_평균단가'].median()
    
    count += 1

In [ ]:
summary_df['차이'] = summary_df['집값중간값'] - summary_df['2종일반중간값']

In [ ]:
summary_df = summary_df.sort_values(['차이'], ascending=False).reset_index(drop=True)
print(summary_df.shape)
summary_df.head()

In [ ]:
summary_df

In [ ]:
summary_df.to_excel('지역별_땅값_일괄매매_중간값.xlsx', index=False)

In [ ]:
summary_df = pd.DataFrame({'지역':[], '2종일반평균':[], '집값평균':[]})

count = 0
for gudong in tqdm(yunrip_bulk_df['구동'].unique()):
    gu = gudong.split('_')[0]
    dong = gudong.split('_')[1]
    
    target_gu = gu
    target_dong = dong
    target_purpose = '2종일반'
    
    target_df = common_df[
        (common_df['구'] == target_gu)
        &(common_df['동'] == target_dong)
        &(common_df['용도지역'].str.contains(target_purpose))
        &(common_df['계약날짜'] >= (date_today - pd.DateOffset(months=target_period)))
        &(common_df['단가(㎡)'] > 250)
    ].sort_values(['번지'])
    
    yunrip_target_df = yunrip_selected_df[
        (yunrip_selected_df['구동'] == gudong)
    ]
    
    summary_df.loc[count, '지역'] = gudong
    summary_df.loc[count, '2종일반평균'] = target_df['단가(㎡)'].mean()
    summary_df.loc[count, '집값평균'] = yunrip_target_df['지번별_평균단가'].mean()
    
    count += 1

In [ ]:
summary_df['차이'] = summary_df['집값평균'] - summary_df['2종일반평균']

In [ ]:
summary_df = summary_df.sort_values(['차이'], ascending=False).reset_index(drop=True)
print(summary_df.shape)
summary_df.head()

In [ ]:
summary_df

In [ ]:
summary_df.to_excel('지역별_땅값_일괄매매_평균.xlsx', index=False)

In [ ]:
yunrip_selected_df.head()

In [ ]:
inv_df = yunrip_selected_df[yunrip_selected_df['구동'] == '성동구_마장동']
print(inv_df.shape)
inv_df.head()

In [ ]:
def mean_prices_per_size(df):
    df = df.copy()
    
    df['크기별_평균단가'] = df[['지번주소', '건축년도', '계약날짜', '전용면적(㎡)', '단가(만원/㎡)']].groupby(['지번주소', '건축년도', '계약날짜', '전용면적(㎡)']).transform('mean')['단가(만원/㎡)']
    df['크기별_평균총액'] = df[['지번주소', '건축년도', '계약날짜', '전용면적(㎡)', '거래금액(만원)']].groupby(['지번주소', '건축년도', '계약날짜', '전용면적(㎡)']).transform('mean')['거래금액(만원)']
    df = df[['지번주소', '건축년도', '건물연식', '계약날짜', '계약날짜기준_건물연식', '전용면적(㎡)', '크기별_평균총액', '크기별_평균단가']]
    df = df.drop_duplicates().reset_index(drop=True)
    
    return df

In [ ]:
yunrip_short_df = mean_prices_per_size(yunrip_selected_df)
print(yunrip_short_df.shape)
yunrip_short_df.head()